In [38]:
import obspy
from obspy.clients.fdsn import Client
import numpy as np
import matplotlib.pyplot as plt
from axikernels.core.handlers import element_output
client = Client("IRIS")

## Importing real data

In [39]:
# Import earthquake data
start_time = obspy.UTCDateTime("2013-05-24T05:44:48")  # Replace with the start time of the earthquake
end_time = obspy.UTCDateTime("2013-05-24T05:44:50")  # Replace with the end time of the earthquake
event = client.get_events(starttime=start_time, endtime=end_time, minmagnitude=8.3)
# Import station data
inventory = client.get_stations(network="BK", station="CMB", channel="BH*", location='00', level="response")
# Import wavefield data
real_data = obspy.read('data/2013-05-24-mw83-sea-of-okhotsk.miniseed')

In [40]:
# Select specific station
selected_stream = real_data.select(station='CMB')
selected_stream.plot()
plt.show()

In [41]:
# Process the station to match the period of the synthetic data
selected_stream.remove_response(inventory=inventory,
                                pre_filt=[1e-4, 1e-3, 0.02, 0.05],
                                output="DISP", water_level=60,
                                plot=True)
selected_stream.detrend()
selected_stream.filter('bandpass', freqmin=0.001, freqmax=0.02)
selected_stream.plot()
plt.show()

## Reproducing the earthquake

We first try to reproduce the wavefield of the real even in a 1D Earth as well as we can. 

1. We use the *prem_ani.bm* basemodel file to create a mesh (anisotropic,   anelastic, but 1D) using the command:

    ```bash
    python -m salvus_mesh_lite.interface AxiSEM -o prem_ani_50s.e --overwrite_file --basic.model model.bm --basic.period 50
    ```
    Then we make sure to use this new mesh in the inparam.model.yaml file.

2. For the inparam.nr.yaml we just need to make sure to use a constant nr=5 in the whole domain since we are doing a 1D simulation (this is where the speed-up happens compared to other full 3D wave solvers).
3. For the source we use the coordinates, depth, abd moment tensor that is found on IRIS. This should give a good enough approximation of the source for our purposes. 
4. Finally, in the inparam.output.yaml we will put 3 element groups: for mantle, outer core (liquid), and inner core. Even though the simulation runs using 16 GLL points on each elements, we will output only 9 for saving memory. 
5. Run axisem3d (the size of the output file will be about 39GB! And the duration is about 0.8 cpu hours on a high end cpu. Maximum RAM usage is about 8GB. Make sure you have sufficient overhead for memory and disk.)



In [52]:
forward_1D_simulation = element_output.ElementOutput('data/1D_SIMULATION/output/elements')
station_coordinates = inventory.get_coordinates('BK.CMB.00.BHZ')
station_point = [6371000 - station_coordinates['elevation'], station_coordinates['latitude'], station_coordinates['longitude']]
forward_1D_simulation.stream(points=[station_point], channels=['UR', 'UT', 'UZ'])

{'latitude': 38.03455, 'longitude': -120.386513, 'elevation': 697.0, 'local_depth': 2.0}
